In [1]:
import spatialdata
import spatialdata_plot
import dask_image.imread
import dask.array as da
import napari_spatialdata
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.colors as mcolors

import geopandas as gpd
import anndata as ad
import numpy as np
import anndata as ad
from loguru import logger

import sys, os
sys.path.append(os.path.abspath("../src/opendvp/"))

for package in [spatialdata, napari_spatialdata]:
    print(f"{package.__name__}: {package.__version__}")

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


spatialdata: 0.3.0
napari_spatialdata: 0.5.5


In [2]:
!mamba list

# packages in environment at /opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata:
#
# Name                    Version                   Build  Channel
aiobotocore               2.5.4                    pypi_0    pypi
aiohappyeyeballs          2.4.6              pyhd8ed1ab_0    conda-forge
aiohttp                   3.11.13         py312h998013c_0    conda-forge
aioitertools              0.12.0                   pypi_0    pypi
aiosignal                 1.3.2              pyhd8ed1ab_0    conda-forge
alabaster                 1.0.0              pyhd8ed1ab_1    conda-forge
anndata                   0.11.3             pyhd8ed1ab_0    conda-forge
annotated-types           0.7.0              pyhd8ed1ab_1    conda-forge
aom                       3.9.1                h7bae524_0    conda-forge
app-model                 0.3.0              pyhd8ed1ab_0    conda-forge
appdirs                   1.4.4              pyhd8ed1ab_1    conda-forge
appnope                   0.1.4              pyhd8ed1ab_

In [3]:
import pandas as pd
import anndata as ad
from loguru import logger 
import time

import numpy as np
import tifffile

import matplotlib.pyplot as plt
import seaborn as sns

import shapely
import geopandas as gpd

from scipy.spatial import Voronoi

# Step 0: Load sdata

In [4]:
sdata = spatialdata.read_zarr("/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/data/sdata/sdata_991.zarr")

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [8]:
bb_xmin = 32000
bb_ymin = 24000
bb_w = 4000
bb_h = 6000
bb_xmax = bb_xmin + bb_w
bb_ymax = bb_ymin + bb_h

cropped_sdata = sdata.query.bounding_box(
    axes=["x", "y"],
    min_coordinate=[bb_xmin, bb_ymin],
    max_coordinate=[bb_xmax, bb_ymax],
    target_coordinate_system="global",
    filter_table=True,
)

In [9]:
cropped_sdata

SpatialData object
├── Images
│     └── 'mIF': DataTree[cyx] (15, 6000, 4000), (15, 2000, 1333), (15, 666, 445), (15, 222, 148)
├── Labels
│     └── 'mask': DataTree[yx] (6000, 4000), (2000, 1333), (666, 445), (222, 148)
├── Shapes
│     ├── 'mask_polygons': GeoDataFrame shape: (13752, 2) (2D shapes)
│     └── 'primary_lmd_contours': GeoDataFrame shape: (24, 8) (2D shapes)
└── Tables
      ├── 'imaging_991': AnnData (13289, 8)
      └── 'proteins': AnnData (12, 4766)
with coordinate systems:
    ▸ 'global', with elements:
        mIF (Images), mask (Labels), mask_polygons (Shapes), primary_lmd_contours (Shapes)

In [10]:
import napari_spatialdata
napari_spatialdata.Interactive(cropped_sdata).run()

# Voronoi

In [13]:
def adataobs_to_voronoi_geojson(
        df,
        imageid, 
        subset:list=None, 
        category_1:str="phenotype", 
        category_2=None, 
        output_path:str="../data/processed/"):

    logger.debug(f" df shape: {df.shape}")

    df = df.copy()
    #subset per image
    df = df[(df.imageid == str(imageid))]
    logger.debug(f" df shape after imageid subset: {df.shape}")
    logger.info(f"Processing {imageid}, loaded dataframe")

    #subset per x,y
    if subset is not None:
        logger.info(f"Subsetting to {subset}")
        assert len(subset) == 4, "subset must be a list of 4 integers"
        x_min, x_max, y_min, y_max = subset
        df = df[(df.X_centroid > x_min) &
                (df.X_centroid < x_max) &
                (df.Y_centroid > y_min) &
                (df.Y_centroid < y_max)]
        #clean subset up
        df = df.reset_index(drop=True)
        if 'Unnamed: 0' in df.columns:
            df.drop(columns=['Unnamed: 0'], inplace=True)

    logger.info("Running Voronoi")
    # run Voronoi 
    # df = df[['X_centroid', 'Y_centroid', category_1, category_2]]    
    vor = Voronoi(df[['X_centroid', 'Y_centroid']].values)
    polygons = []
    for i in range(len(df)):
        polygon = shapely.Polygon(
            [vor.vertices[vertex] for vertex in vor.regions[vor.point_region[i]]])
        polygons.append(polygon)
    df['geometry'] = polygons
    logger.info("Voronoi done")

    #transform to geodataframe
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    logger.info("Transformed to geodataframe")

    # filter polygons that go outside of image
    if subset is None:
        x_min = gdf['X_centroid'].min()
        x_max = gdf['X_centroid'].max()
        y_min = gdf['Y_centroid'].min()
        y_max = gdf['Y_centroid'].max()
        logger.info(f"Bounding box: x_min: {x_min}, x_max: {x_max}, y_min: {y_min}, y_max {y_max}")

    boundary_box = shapely.box(x_min, y_min, x_max, y_max)
    gdf = gdf[gdf.geometry.apply(lambda poly: poly.within(boundary_box))]
    logger.info("Filtered out infinite polygons")

    # filter polygons that are too large
    gdf['area'] = gdf['geometry'].area
    gdf = gdf[gdf['area'] < gdf['area'].quantile(0.98)]
    logger.info("Filtered out large polygons based on the 98% quantile")
    # filter polygons that are very pointy
    
    # TODO improve filtering by pointiness
    # gdf = process_polygons(gdf, scale_threshold=350, remove_threshold=400, scale_factor=0.3)
    # logger.info("Filtered out pointy polygons")

    # create geodataframe for each cell and their celltype
    gdf2 = gdf.copy()
    gdf2['objectType'] = 'cell'
    gdf2['classification'] = gdf2[category_1]
    
    # merge polygons based on the CN column
    if category_2:
        logger.info("Merging polygons for cellular neighborhoods")
        gdf3 = gdf.copy()
        gdf3 = gdf3.dissolve(by=category_2)
        gdf3[category_2] = gdf3.index
        gdf3 = gdf3.explode(index_parts=True)
        gdf3 = gdf3.reset_index(drop=True)
        gdf3['classification'] = gdf3[category_2].astype(str)
        
    #export to geojson
    datetime = time.strftime("%Y%m%d_%H%M")
    gdf2.to_file(f"{output_path}/{datetime}_{imageid}_cells_voronoi.geojson", driver='GeoJSON')
    if category_2:
        gdf3.to_file(f"{output_path}/{datetime}_{imageid}_RCN_voronoi.geojson", driver='GeoJSON')

    logger.success(f"Exported {imageid} to geojson")

In [5]:
sdata['imaging_991'].obs

,CellID,Y_centroid,X_centroid,Area,MajorAxisLength,MinorAxisLength,Eccentricity,Orientation,Extent,Solidity,...,imageid,phenotype,cell_id,spatial_lda_knn7_kmeans_k7,spatial_lda_knn14_kmeans_k7,spatial_lda_knn21_kmeans_k7,spatial_lda_knn30_kmeans_k7,spatial_lda_knn40_kmeans_k7,spatial_lda_knn50_kmeans_k7,shapes
0,1,29410.806452,35874.857801,1519.0,98.696620,30.646839,0.950568,-1.524977,228.651804,0.676615,...,991,Cancer_cells,991_1,0,3,2,5,3,6,mask_polygons
1,2,29445.080408,36162.243140,1567.0,50.422862,39.687556,0.616832,0.160002,148.710678,0.975109,...,991,Cancer_cells,991_2,2,3,2,5,3,6,mask_polygons
2,3,29505.171707,38455.235122,1025.0,40.280587,34.195207,0.528513,-0.346644,131.438600,0.927602,...,991,Cancer_cells,991_3,0,0,1,0,3,6,mask_polygons
3,4,29539.721673,35060.844867,1315.0,61.049935,28.208869,0.886848,-1.447088,151.254834,0.947406,...,991,CD4_Tcells,991_4,3,5,0,2,4,2,mask_polygons
4,5,29982.642779,36765.660558,3656.0,84.178897,57.384893,0.731630,0.251203,243.379726,0.926743,...,991,Cancer_cells,991_5,0,0,1,0,1,3,mask_polygons
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685800,685801,40389.067593,46087.962963,1080.0,56.372404,25.352581,0.893163,1.002754,141.610173,0.961710,...,991,COL1A1_cells,991_685801,1,5,0,2,4,4,mask_polygons
685801,685802,40382.629182,47716.017658,1076.0,41.856049,34.076776,0.580666,1.330143,129.396970,0.951370,...,991,Cancer_cells,991_685802,0,0,1,0,1,3,mask_polygons
685802,685803,40382.072917,48436.798177,768.0,33.256837,29.969709,0.433488,0.983058,105.254834,0.957606,...,991,CD8_Tcells,991_685803,1,5,6,6,0,5,mask_polygons
685803,685804,40383.786948,45011.021113,1042.0,45.051210,31.924524,0.705583,1.218499,133.189863,0.964815,...,991,Unknown,991_685804,6,6,0,6,4,2,mask_polygons


In [15]:
sdata['imaging_991'].obs.columns

Index(['CellID', 'Y_centroid', 'X_centroid', 'Area', 'MajorAxisLength',
       'MinorAxisLength', 'Eccentricity', 'Orientation', 'Extent', 'Solidity',
       'artefact', 'Area_filter_nottoobig', 'Area_filter_nottoolow',
       'Area_filter', 'DAPI_ratio', 'DAPI_ratio_pass_nottoolow',
       'DAPI_ratio_pass_nottoohigh', 'DAPI_ratio_pass', 'filtering', 'imageid',
       'phenotype', 'cell_id', 'spatial_lda_knn7_kmeans_k7',
       'spatial_lda_knn14_kmeans_k7', 'spatial_lda_knn21_kmeans_k7',
       'spatial_lda_knn30_kmeans_k7', 'spatial_lda_knn40_kmeans_k7',
       'spatial_lda_knn50_kmeans_k7', 'shapes'],
      dtype='object')

In [36]:
RCN = "spatial_lda_knn21_kmeans_k7"

In [53]:
df = sdata['imaging_991'].obs[['X_centroid', "Y_centroid", RCN]]

In [54]:
vor = Voronoi(df[['X_centroid', 'Y_centroid']].values)
polygons = []
for i in range(len(df)):
    polygon = shapely.Polygon(
        [vor.vertices[vertex] for vertex in vor.regions[vor.point_region[i]]])
    polygons.append(polygon)
df['geometry'] = polygons
logger.info("Voronoi done")

/var/folders/x7/grkjlk8s223dy6234rnz1885mxz2_6/T/ipykernel_27832/3987500421.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = polygons
2025-04-02 15:16:58.355 | INFO     | __main__:<module>:8 - Voronoi done


In [55]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [56]:
# filter polygons that go outside of image
x_min = gdf['X_centroid'].min()
x_max = gdf['X_centroid'].max()
y_min = gdf['Y_centroid'].min()
y_max = gdf['Y_centroid'].max()
logger.info(f"Bounding box: x_min: {x_min}, x_max: {x_max}, y_min: {y_min}, y_max {y_max}")

boundary_box = shapely.box(x_min, y_min, x_max, y_max)
gdf = gdf[gdf.geometry.apply(lambda poly: poly.within(boundary_box))]
logger.info("Filtered out infinite polygons")

2025-04-02 15:16:58.417 | INFO     | __main__:<module>:6 - Bounding box: x_min: 878.0318021201414, x_max: 71605.30902111324, y_min: 6.78003120124805, y_max 42764.688888888886


2025-04-02 15:17:03.077 | INFO     | __main__:<module>:10 - Filtered out infinite polygons


In [57]:
RCN_colors = {
    "0" : "#64b5cd", #cyan
    "1" : "#c44e52", #red
    "2" : "#da8bc3", #pink
    "3" : "#ccb974", #ocre
    "4" : "#8c8c8c", #grey
    "5" : "#dd8452", #orange
    "6" : "#4c72b0", #blue
    "filtered_out" : "#000000" #black
}

In [58]:
import re

In [59]:
def parse_color_for_qupath(color_dict):
    parsed_colors = {}
    for name, color in color_dict.items():
        if isinstance(color, tuple) and len(color) == 3:
            # Handle RGB fraction tuples (0-1)
            parsed_colors[name] = list(int(c * 255) for c in color)
        elif isinstance(color, str) and re.match(r'^#(?:[0-9a-fA-F]{3}){1,2}$', color):
            # Handle hex codes
            parsed_colors[name] = mcolors.hex2color(color)
            parsed_colors[name] = list(int(c * 255) for c in parsed_colors[name])
        else:
            raise ValueError(f"Invalid color format for '{name}': {color}")
        
    return parsed_colors

color_dict = parse_color_for_qupath(RCN_colors)

In [60]:
color_dict

{'0': [100, 181, 205],
 '1': [196, 78, 82],
 '2': [218, 139, 195],
 '3': [204, 185, 116],
 '4': [140, 140, 140],
 '5': [221, 132, 82],
 '6': [76, 114, 176],
 'filtered_out': [0, 0, 0]}

In [61]:
gdf['classification'] = gdf.apply(lambda x: {'name': x[RCN], 'color': color_dict[x[RCN]]}, axis=1)

In [64]:
gdf.head()

,X_centroid,Y_centroid,spatial_lda_knn21_kmeans_k7,geometry,classification
0,35874.857801,29410.806452,2,"POLYGON ((35921.482 29405.399, 35855.526 29488...","{'name': '2', 'color': [218, 139, 195]}"
1,36162.243140,29445.080408,2,"POLYGON ((36188.822 29484.306, 36211.088 29433...","{'name': '2', 'color': [218, 139, 195]}"
2,38455.235122,29505.171707,1,"POLYGON ((38461.607 29485.782, 38478.09 29500....","{'name': '1', 'color': [196, 78, 82]}"
3,35060.844867,29539.721673,0,"POLYGON ((35076.351 29526.462, 35086.513 29536...","{'name': '0', 'color': [100, 181, 205]}"
4,36765.660558,29982.642779,1,"POLYGON ((36771.474 29938.975, 36799.839 30014...","{'name': '1', 'color': [196, 78, 82]}"


In [66]:
gdf3.spatial_lda_knn21_kmeans_k7.dtype

CategoricalDtype(categories=['0', '1', '2', '3', '4', '5', '6'], ordered=False, categories_dtype=object)

In [67]:
gdf3 = gdf.copy()
gdf3 = gdf3.dissolve(by=RCN)
# gdf3[RCN] = gdf3.index
# gdf3 = gdf3.explode(index_parts=True)
# gdf3 = gdf3.reset_index(drop=True)
# gdf3['classification'] = gdf3[RCN].astype(str)

GEOSException: TopologyException: side location conflict at 67950.625975169853 5423.3885138868864. This can occur if the input geometry is invalid.

In [68]:
print(gdf3[~gdf3.is_valid]) 

         X_centroid   Y_centroid spatial_lda_knn21_kmeans_k7  \
20360  67143.384483  5575.721839                           4   

                                                geometry  \
20360  POLYGON ((69747.888 2756.114, 67730.492 4484.8...   

                                classification  
20360  {'name': '4', 'color': [140, 140, 140]}  


In [69]:
gdf3["geometry"] = gdf3["geometry"].buffer(0)
print(gdf3[~gdf3.is_valid]) 

Empty GeoDataFrame
Columns: [X_centroid, Y_centroid, spatial_lda_knn21_kmeans_k7, geometry, classification]
Index: []


In [ ]:
gdf3 = gdf.copy()
gdf3["geometry"] = gdf3["geometry"].buffer(0)
gdf3 = gdf3.dissolve(by=RCN)

In [74]:
gdf3 = gdf.copy()
gdf3["geometry"] = gdf3["geometry"].buffer(0)
gdf3 = gdf3.dissolve(by=RCN)

gdf3[RCN] = gdf3.index
gdf3 = gdf3.explode(index_parts=True)
gdf3 = gdf3.reset_index(drop=True)
gdf3['classification'] = gdf3.apply(lambda x: {'name': x[RCN], 'color': color_dict[x[RCN]]}, axis=1)

In [75]:
gdf3

,X_centroid,Y_centroid,classification,spatial_lda_knn21_kmeans_k7,geometry
0,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}",0,"POLYGON ((17235.32 21813.756, 17165.284 21829...."
1,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}",0,"POLYGON ((26008.09 22093.624, 25989.496 22087...."
2,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}",0,"POLYGON ((25857.976 22482.543, 25836.587 22509..."
3,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}",0,"POLYGON ((26368.226 26237.121, 26358.081 26261..."
4,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}",0,"POLYGON ((19705.103 26082.93, 19707.441 26106...."
...,...,...,...,...,...
20140,34516.052557,33618.546875,"{'name': '6', 'color': [76, 114, 176]}",6,"POLYGON ((64888.901 32299.665, 64935.477 32293..."
20141,34516.052557,33618.546875,"{'name': '6', 'color': [76, 114, 176]}",6,"POLYGON ((65077.578 32428.386, 65069.486 32454..."
20142,34516.052557,33618.546875,"{'name': '6', 'color': [76, 114, 176]}",6,"POLYGON ((65159.247 32681.572, 65171.895 32642..."
20143,34516.052557,33618.546875,"{'name': '6', 'color': [76, 114, 176]}",6,"POLYGON ((66072.188 33516.584, 66079.446 33512..."


In [77]:
# remove class column to keep clean
gdf3.drop(columns=RCN, inplace=True)

In [78]:
gdf3['objectType'] = "detection"

In [79]:
gdf3.head()

,X_centroid,Y_centroid,classification,geometry,objectType
0,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}","POLYGON ((17235.32 21813.756, 17165.284 21829....",detection
1,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}","POLYGON ((26008.09 22093.624, 25989.496 22087....",detection
2,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}","POLYGON ((25857.976 22482.543, 25836.587 22509...",detection
3,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}","POLYGON ((26368.226 26237.121, 26358.081 26261...",detection
4,35060.844867,29539.721673,"{'name': '0', 'color': [100, 181, 205]}","POLYGON ((19705.103 26082.93, 19707.441 26106....",detection


In [80]:
gdf3.to_file("../data/try4_voronoi.geojson", driver='GeoJSON')    

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [81]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/1_Pipelines/openDVP/data/sdata/sdata_991.zarr
├── Images
│     └── 'mIF': DataTree[cyx] (15, 44470, 73167), (15, 14823, 24389), (15, 4941, 8129), (15, 1647, 2709)
├── Labels
│     └── 'mask': DataTree[yx] (44470, 73167), (14823, 24389), (4941, 8129), (1647, 2709)
├── Shapes
│     ├── 'mask_polygons': GeoDataFrame shape: (685805, 2) (2D shapes)
│     ├── 'primary_lmd_contours': GeoDataFrame shape: (234, 8) (2D shapes)
│     └── 'relapse_lmd_contours': GeoDataFrame shape: (214, 8) (2D shapes)
└── Tables
      ├── 'imaging_991': AnnData (610182, 8)
      └── 'proteins': AnnData (219, 4766)
with coordinate systems:
    ▸ 'global', with elements:
        mIF (Images), mask (Labels), mask_polygons (Shapes), primary_lmd_contours (Shapes), relapse_lmd_contours (Shapes)

In [82]:
mask_polygons = sdata['mask_polygons'].copy()

In [83]:
mask_polygons

,label,geometry
label,,
1,1,"POLYGON ((35840.5 29406, 35840 29405.5, 35839...."
2,2,"POLYGON ((36181.5 29440, 36181.5 29439, 36181 ..."
3,3,"POLYGON ((38461 29530.5, 38460 29530.5, 38459...."
4,4,"POLYGON ((35073 29552.5, 35074 29552.5, 35075 ..."
5,5,"POLYGON ((36786.5 29952, 36786.5 29951, 36786 ..."
...,...,...
685801,685801,"POLYGON ((46080.5 40372, 46080 40371.5, 46079 ..."
685802,685802,"POLYGON ((47713 40404.5, 47712 40404.5, 47711...."
685803,685803,"POLYGON ((48446 40398.5, 48445 40398.5, 48444 ..."


In [84]:
mask_polygons['objectType'] = "detection"

In [89]:
mask_polygons['geometry'] = mask_polygons['geometry'].simplify(1, preserve_topology=True)
mask_polygons = mask_polygons.drop(columns='label', inplace=False) 

In [90]:
mask_polygons.to_file("../data/just_seg_v2.geojson", driver="GeoJSON")

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [88]:
if 'label' in mask_polygons.columns: # sdata.to_polygonize creates double label column, we drop it
        gdf_tmp = mask_polygons.drop(columns='label', inplace=False) 
        gdf_tmp.to_file("../data/just_seg.geojson", driver="GeoJSON")

KeyboardInterrupt: 

In [87]:
mask_polygons.head()

,label,geometry,objectType
label,,,
1,1,"POLYGON ((35840.5 29406, 35840 29405.5, 35839....",detection
2,2,"POLYGON ((36181.5 29440, 36181.5 29439, 36181 ...",detection
3,3,"POLYGON ((38461 29530.5, 38460 29530.5, 38459....",detection
4,4,"POLYGON ((35073 29552.5, 35074 29552.5, 35075 ...",detection
5,5,"POLYGON ((36786.5 29952, 36786.5 29951, 36786 ...",detection
